# Explainability Agent

zu klären:
- lime: as_list
- shap: https://github.com/slundberg/shap/issues/632
- ebm Output = Feature importance?

Goal: nur model, welches wir am Ende nehmen, erklären

Overall: nach activity agent bestes model (accuracy) hier rein, kommt aus Kai's Liste für verschiedene Households --> feature importance als input für recommendation

Recommendation verlängern: Unsere Empfehlung sprechen wir aus, weil wir denken, dass du heute den tumble dryer benutzt, weil 3 höchste feature importances. 

Features in Lesbares umwandeln, z.B. activity_lag_1 --> Du warst in den letzten 24 Stunden zuhause.
--> Wenn aber alle 3 letzten Features activity_lags sind, nehmen wir die 4. feature importance statt der 2. und 3. oder so

Plot: 
- shap: feature importance: image = True
- lime: 

Recommendation an sich: 
- relative savings aus Alonas Paper?

In [ ]:
# Explainability Agent
# ===============================================================================================
class Explainability_Agent:
    def __init__(self, model_activity, X_train_activity, X_test_activity, best_hour, model_usage,
               X_train_usage, X_test_usage, model_type="logit"):
        self.model_activity = model_activity
        self.model_type = model_type
        self.X_train_activity = X_train_activity
        self.X_test_activity = X_test_activity
        self.best_hour = best_hour
        self.model_usage = model_usage
        self.X_train_usage = X_train_usage
        self.X_test_usage = X_test_usage

    def feature_importance(self):
        print(self.model_type)

        if self.model_type == "logit":
            X_train_summary = shap.kmeans(self.X_train_activity, 10)
            explainer = shap.KernelExplainer(self.model_activity.predict_proba, X_train_summary)

        elif self.model_type == "ada":
            X_train_summary = shap.kmeans(self.X_train_activity, 10)
            explainer = shap.KernelExplainer(self.model_activity.predict_proba, X_train_summary)

        elif self.model_type == "knn":
            X_train_summary = shap.kmeans(self.X_train_activity, 10)
            explainer = shap.KernelExplainer(self.model_activity.predict_proba, X_train_summary)

        elif self.model_type == "random forest":
            explainer = shap.TreeExplainer(self.model_activity, self.X_train_activity)

        elif self.model_type == "xgboost":
            explainer = shap.TreeExplainer(self.model_activity, self.X_train_activity, model_output='predict_proba')
        else:
            raise InputError("Unknown model type.")

        shap_values = explainer.shap_values(
            self.X_test_activity.iloc[self.best_hour, :])

        feature_names_activity = list(self.X_train_activity.columns.values)
        feature_names_activity
        # %%
        vals_activity = np.abs(shap_values).mean(0)
        # vals_activity = shap_values[1]
        # %%
        feature_importance_activity = pd.DataFrame(list(zip(feature_names_activity, vals_activity)),
                                                   columns=['col_name', 'feature_importance_vals'])
        feature_importance_activity.sort_values(by=['feature_importance_vals'], ascending=False, inplace=True)
        feature_importance_activity

        # usage
        if self.model_type == "logit":
            # option: apply kmeans first for faster computation
            X_train_summary = shap.kmeans(self.X_train_usage, 10)
            explainer = shap.KernelExplainer(self.model_usage.predict_proba, X_train_summary)
            # without kmeans:
            # explainer = shap.KernelExplainer(model.predict_proba, X_train)

        elif self.model_type == "ada":
            X_train_summary = shap.kmeans(self.X_train_usage, 10)
            explainer = shap.KernelExplainer(self.model_usage.predict_proba, X_train_summary)

        elif self.model_type == "knn":
            X_train_summary = shap.kmeans(self.X_train_usage, 10)
            explainer = shap.KernelExplainer(self.model_usage.predict_proba, X_train_summary)

        elif self.model_type == "random forest":
            explainer = shap.TreeExplainer(self.model_usage, self.X_train_usage)

        elif self.model_type == "xgboost":
            explainer = shap.TreeExplainer(self.model_usage, self.X_train_usage, model_output='predict_proba')
        else:
            raise InputError("Unknown model type.")

        shap_values_usage = explainer.shap_values(
            self.X_test_usage)

        feature_names_usage = list(self.X_train_usage.columns.values)
        feature_names_usage
        # %%
        vals = np.abs(shap_values_usage).mean(0)
        # vals = shap_values[1]
        # %%
        feature_importance_usage = pd.DataFrame(list(zip(feature_names_usage, vals)),
                                                columns=['col_name', 'feature_importance_vals'])
        feature_importance_usage.sort_values(by=['feature_importance_vals'], ascending=False, inplace=True)
        feature_importance_usage

        return feature_importance_activity, feature_importance_usage